In [4]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box

# NEW imports for extra models & metrics
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

import osmnx as ox
import folium
import matplotlib.pyplot as plt

# =========================
# CONFIG
# =========================
POA_FILE = "output/stages/06_poa_demographics_summary.csv"
CHARGER_FILE = "output/stages/02_chargers_active_sample.csv"
OUT_DIR = "output/grid_ml"
os.makedirs(OUT_DIR, exist_ok=True)

SYDNEY_BBOX = (150.5, -34.2, 151.5, -33.5)  # lon/lat
GRID_CELL_SIZE_M = 1000

# Split features to avoid leakage:
# - Classifier: NO 'total_plugs' (proxy for label)
# - Regressor: can include 'total_plugs' as a feature when fitting on has_plugs==1 rows
POA_FEATURES_CLASSIFIER = [
    "population", "car_commuters", "ev_count",
    "total_vehicles", "ev_proportion", "traffic_count"
]
POA_FEATURES_REGRESSOR = [
    "population", "car_commuters", "ev_count",
    "total_vehicles", "ev_proportion", "traffic_count"
]

# =========================
# LOAD DATA
# =========================
def load_poa():
    df = pd.read_csv(POA_FILE)
    # Expect columns: POA_CODE, centroid_lon, centroid_lat, has_plugs, total_plugs, and the feature columns above
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["centroid_lon"], df["centroid_lat"]),
        crs="EPSG:4326"
    )
    print(f"Loaded POA dataset ({len(gdf)} rows)")
    return gdf

def load_chargers():
    df = pd.read_csv(CHARGER_FILE)
    # Expect columns named latitude / longitude (adjust if your file differs)
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
        crs="EPSG:4326"
    )
    print(f"Loaded charger dataset ({len(gdf)} rows)")
    return gdf

# =========================
# MODELS @ POA LEVEL
# =========================
def train_poa_classifier(poa_gdf):
    """
    Compare 3 classifiers:
      - RandomForestClassifier (balanced)
      - LogisticRegression (balanced)
      - XGBClassifier (scale_pos_weight)
    Select best by ROC-AUC (tie-break by F1).
    Save poa_scored.csv with 'need_score' from best model.
    """
    X = poa_gdf[POA_FEATURES_CLASSIFIER].fillna(0)
    y = poa_gdf["has_plugs"].astype(int)

    # Guard against single-class issues
    if y.nunique() < 2:
        raise ValueError("has_plugs must contain both 0 and 1 to train a classifier.")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # class imbalance helper
    pos_ratio = (len(y_train) - y_train.sum()) / max(y_train.sum(), 1)

    models = {
        "RandomForest": RandomForestClassifier(
            n_estimators=300, max_depth=None, class_weight="balanced", random_state=42
        ),
        "LogisticRegression": LogisticRegression(
            solver="liblinear", class_weight="balanced", random_state=42
        ),
        "XGBoost": xgb.XGBClassifier(
            objective="binary:logistic",
            n_estimators=300,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            random_state=42,
            eval_metric="auc",
            scale_pos_weight=float(pos_ratio) if pos_ratio > 0 else 1.0
        )
    }

    rows = []
    fitted = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        fitted[name] = model

        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)[:, 1]
        else:
            # For models without predict_proba (not the case here), fallback:
            y_prob = model.predict(X_test)
            # ensure probabilities are in [0,1]
            y_prob = np.clip(y_prob.astype(float), 0, 1)

        y_pred = (y_prob >= 0.5).astype(int)

        row = {
            "model": name,
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, zero_division=0),
            "recall": recall_score(y_test, y_pred, zero_division=0),
            "f1": f1_score(y_test, y_pred, zero_division=0),
            "roc_auc": roc_auc_score(y_test, y_prob)
        }
        rows.append(row)

    perf_df = pd.DataFrame(rows).sort_values(["roc_auc", "f1"], ascending=[False, False])
    print("\nPOA-Level Classifier Performance (comparison):")
    print(perf_df.to_string(index=False))

    # choose best by ROC-AUC, tie-break F1
    best_name = perf_df.iloc[0]["model"]
    clf = fitted[best_name]
    print(f"\n✅ Selected classifier: {best_name}")

    # need_score from best model
    need_prob = clf.predict_proba(X)[:, 1] if hasattr(clf, "predict_proba") else clf.predict(X)
    poa_gdf["need_score"] = need_prob

    # Save scored POA
    save_path = os.path.join(OUT_DIR, "poa_scored.csv")
    poa_gdf.to_csv(save_path, index=False)
    print(f"✅ Saved POA scored dataset to: {save_path}")

    # Also save model comparison for reporting
    perf_df.to_csv(os.path.join(OUT_DIR, "classifier_model_comparison.csv"), index=False)

    return clf, poa_gdf

def train_capacity_regressor(poa_gdf):
    """
    Compare 3 regressors on POAs with existing chargers:
      - XGBRegressor
      - RandomForestRegressor
      - Ridge (linear)
    Select best by RMSE (tie-break MAE).
    Return best regressor + scaler.
    """
    train = poa_gdf[poa_gdf["has_plugs"] == 1].copy()
    if len(train) < 5:
        raise ValueError("Too few positive POAs to train capacity regressor.")

    X = train[POA_FEATURES_REGRESSOR].fillna(0)
    y = train["total_plugs"].clip(lower=1)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

    # Use one scaler for linear model and to keep your downstream API the same
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    models = {
        "XGBRegressor": xgb.XGBRegressor(
            objective="reg:squarederror",
            n_estimators=300,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42
        ),
        "RandomForestRegressor": RandomForestRegressor(
            n_estimators=400, max_depth=None, random_state=42, n_jobs=-1
        ),
        "Ridge": Ridge(alpha=1.0, random_state=42)
    }

    rows = []
    fitted = {}
    for name, model in models.items():
        # tree models don't need scaling; ridge does
        if name in ("XGBRegressor", "RandomForestRegressor"):
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        else:
            model.fit(X_train_s, y_train)
            y_pred = model.predict(X_test_s)

        row = {
            "model": name,
            "rmse": float(np.sqrt(mean_squared_error(y_test, y_pred))),
            "mae": float(mean_absolute_error(y_test, y_pred)),
            "r2": float(r2_score(y_test, y_pred))
        }
        rows.append(row)
        fitted[name] = model

    perf_df = pd.DataFrame(rows).sort_values(["rmse", "mae"], ascending=[True, True])
    print("\nPOA-Level Capacity Regressor Performance (comparison):")
    print(perf_df.to_string(index=False))

    best_name = perf_df.iloc[0]["model"]
    reg = fitted[best_name]
    print(f"\n✅ Selected regressor: {best_name}")

    # Save model comparison for reporting
    perf_df.to_csv(os.path.join(OUT_DIR, "regressor_model_comparison.csv"), index=False)

    return reg, scaler

# =========================
# GRID + MERGE POA FEATURES
# =========================
def create_grid(bbox, cell_size_m, gdf_poa):
    """
    Create square grid over bbox and CLIP it to NSW POA boundaries.
    This ensures we only keep grid cells that fall within NSW.
    """
    minx, miny, maxx, maxy = bbox
    bbox_poly = box(minx, miny, maxx, maxy)

    bbox_gdf = gpd.GeoDataFrame(geometry=[bbox_poly], crs="EPSG:4326").to_crs(3857)
    minx_m, miny_m, maxx_m, maxy_m = bbox_gdf.total_bounds

    xs = np.arange(minx_m, maxx_m, cell_size_m)
    ys = np.arange(miny_m, maxy_m, cell_size_m)

    cells = [box(x, y, x + cell_size_m, y + cell_size_m) for x in xs for y in ys]

    grid = gpd.GeoDataFrame({"cell_id": range(len(cells))}, geometry=cells, crs="EPSG:3857").to_crs(4326)
    print(f"Grid cells generated before clipping: {len(grid)}")

    gdf_poa = gdf_poa.to_crs(4326)
    grid_clipped = gpd.sjoin(
        grid,
        gdf_poa[["geometry"]],
        how="inner",
        predicate="intersects"
    ).drop(columns=["index_right"]).reset_index(drop=True)

    print(f"✅ Grid cells retained after clipping to NSW POA: {len(grid_clipped)}")
    return grid_clipped

def assign_poa_to_grid(grid, poa):
    # Nearest POA centroid to each grid cell centroid
    poa_pts = poa.copy().to_crs(3857)
    grid_m = grid.to_crs(3857)

    from scipy.spatial import cKDTree
    tree = cKDTree(np.vstack([poa_pts.geometry.x, poa_pts.geometry.y]).T)
    centroids = np.vstack([grid_m.geometry.centroid.x, grid_m.geometry.centroid.y]).T
    _, idx = tree.query(centroids, k=1)

    grid["assigned_poa"] = poa_pts.iloc[idx]["POA_CODE"].values
    return grid

def merge_poa_features_into_grid(grid, poa_scored):
    # Keep only the needed columns
    keep = ["POA_CODE", "need_score"] + sorted(set(POA_FEATURES_CLASSIFIER + POA_FEATURES_REGRESSOR))
    keep = [c for c in keep if c in poa_scored.columns]
    grid = grid.merge(poa_scored[keep], left_on="assigned_poa", right_on="POA_CODE", how="left")

    # Fill with means to avoid NaNs
    for col in keep:
        if col not in ["POA_CODE"]:
            grid[col] = grid[col].fillna(grid[col].mean())

    return grid

# =========================
# OSM POIs (safe)
# =========================
def fetch_pois(grid):
    print("Downloading OSM POIs (safe mode)...")
    try:
        from shapely.geometry import box as _box
        minx, miny, maxx, maxy = grid.total_bounds
        poly = _box(minx, miny, maxx, maxy)

        pois = ox.features_from_polygon(poly, {"amenity": True})
        if pois.empty:
            print("⚠️ No POIs found. Setting poi_count = 0")
            grid["poi_count"] = 0
            return grid

        pois = pois.to_crs(4326).copy()
        pois["geometry"] = pois.geometry.centroid

        join = gpd.sjoin(
            pois[["geometry"]],
            grid[["cell_id", "geometry"]],
            how="left",
            predicate="within"
        )

        counts = join.groupby("cell_id").size().rename("poi_count").reset_index()
        grid = grid.merge(counts, on="cell_id", how="left")
        grid["poi_count"] = grid["poi_count"].fillna(0).astype(int)

    except Exception as e:
        print("⚠️ POI fetch failed, setting poi_count = 0. Reason:", e)
        grid["poi_count"] = 0

    return grid

# =========================
# SCORING & OUTPUT
# =========================
def score_cells(grid):
    # Simple composite: demand * (POIs + 1)
    grid["final_score"] = grid["need_score"] * (grid["poi_count"] + 1)
    return grid.sort_values("final_score", ascending=False)
    
def plot_top_final_scores(grid, out_dir=OUT_DIR):
    """
    Create a bar chart showing the top 20 NSW grid cells (or suburbs)
    ranked by final_score (charger need intensity).
    """
    top20 = grid.sort_values("final_score", ascending=False).head(20).copy()

    plt.figure(figsize=(10, 6))
    plt.barh(
        top20["assigned_poa"].astype(str),
        top20["final_score"],
        color="#ff6f3c"
    )
    plt.gca().invert_yaxis()  # Highest at top
    plt.title("Top 20 NSW Locations by Predicted EV Charger Need (Final Score)", fontsize=14, pad=15)
    plt.xlabel("Final Score (Need Intensity)")
    plt.ylabel("Postcode Area (POA)")
    plt.grid(axis="x", linestyle="--", alpha=0.6)
    plt.tight_layout()

    output_path = os.path.join(out_dir, "top20_final_score.png")
    plt.savefig(output_path, dpi=300)
    plt.close()
    print(f"✅ Saved Top 20 Final Score plot: {output_path}")

def run_pipeline():
    poa = load_poa()
    _chargers = load_chargers()  # not used directly yet, but kept for future features

    # 1) Train POA models (now compares 3 & selects best)
    clf, poa_scored = train_poa_classifier(poa)

    # 2) Train capacity models (now compares 3 & selects best)
    reg, scaler = train_capacity_regressor(poa)

    # 3) Grid + tie to nearest POA
    grid = create_grid(SYDNEY_BBOX, GRID_CELL_SIZE_M, poa)
    grid = assign_poa_to_grid(grid, poa)
    grid = merge_poa_features_into_grid(grid, poa_scored)

    # 4) POIs
    grid = fetch_pois(grid)

    # 5) Score locations
    grid = score_cells(grid)

    # 6) Predict capacity per grid cell using the selected regressor
    reg_feat_in_grid = [c for c in POA_FEATURES_REGRESSOR if c in grid.columns]
    X_grid = grid[reg_feat_in_grid].fillna(0)
    X_grid = scaler.transform(X_grid)

    preds = reg.predict(X_grid)
    grid["pred_plugs"] = np.clip(np.round(preds).astype(int), 1, None)

    # 7) Save results
    out_csv = os.path.join(OUT_DIR, "recommended_candidates.csv")
    grid.to_csv(out_csv, index=False)
    print("\n✅ Saved:", out_csv)
      # 🔹 Generate visualisation
    plot_top_final_scores(grid)

def plot_model_comparisons():
    """
    Create bar charts for classifier and regressor model comparisons.
    Reads the CSVs created earlier and saves plots in output/grid_ml/.
    """
    clf_path = os.path.join(OUT_DIR, "classifier_model_comparison.csv")
    reg_path = os.path.join(OUT_DIR, "regressor_model_comparison.csv")

    # --- Classification models ---
    if os.path.exists(clf_path):
        df_clf = pd.read_csv(clf_path)
        metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]

        plt.figure(figsize=(8, 5))
        df_clf.set_index("model")[metrics].plot(kind="bar", figsize=(8, 5))
        plt.title("POA Classifier Model Comparison")
        plt.ylabel("Score")
        plt.ylim(0, 1)
        plt.xticks(rotation=0)
        plt.legend(loc="lower right")
        plt.grid(axis="y", linestyle="--", alpha=0.6)
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "classifier_model_comparison.png"), dpi=300)
        plt.close()
        print("✅ Saved classifier comparison chart")

    # --- Regression models ---
    if os.path.exists(reg_path):
        df_reg = pd.read_csv(reg_path)
        metrics = ["rmse", "mae", "r2"]

        plt.figure(figsize=(8, 5))
        df_reg.set_index("model")[metrics].plot(kind="bar", figsize=(8, 5))
        plt.title("POA Capacity Regressor Model Comparison")
        plt.ylabel("Metric Value")
        plt.xticks(rotation=0)
        plt.grid(axis="y", linestyle="--", alpha=0.6)
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "regressor_model_comparison.png"), dpi=300)
        plt.close()
        print("✅ Saved regressor comparison chart")

# Add this line after run_pipeline() completes
if __name__ == "__main__":
    run_pipeline()
    plot_model_comparisons()

Loaded POA dataset (612 rows)
Loaded charger dataset (1426 rows)

POA-Level Classifier Performance (comparison):
             model  accuracy  precision   recall       f1  roc_auc
      RandomForest  0.755435   0.801802 0.794643 0.798206 0.795573
           XGBoost  0.722826   0.796117 0.732143 0.762791 0.787822
LogisticRegression  0.739130   0.771186 0.812500 0.791304 0.782986

✅ Selected classifier: RandomForest
✅ Saved POA scored dataset to: output/grid_ml/poa_scored.csv

POA-Level Capacity Regressor Performance (comparison):
                model      rmse      mae       r2
                Ridge 12.656334 8.046490 0.433451
RandomForestRegressor 14.903844 9.003817 0.214370
         XGBRegressor 16.136690 9.655774 0.079019

✅ Selected regressor: Ridge
Grid cells generated before clipping: 10528
✅ Grid cells retained after clipping to NSW POA: 235

✅ Saved: output/grid_ml/recommended_candidates.csv
✅ Saved Top 20 Final Score plot: output/grid_ml/top20_final_score.png
✅ Saved classifie

<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>